In [ ]:
#!/usr/bin/python3
# coding: utf-8
# Hokkaido

In [ ]:
import codecs
from datetime import datetime as dt
from datetime import timedelta as td
from jma_csvdl import save_jma_data, parse_jma_csv
import json
import sys
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
import time
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly, make_exp_fit_graph

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')
ogiwara_uri = "https://toyokeizai.net/sp/visual/tko/covid19/csv/"
pref_file = "prefectures.csv"
download_if_needed(ogiwara_uri, pref_file)

In [ ]:
populations = get_populations()

In [ ]:
offsets = dict(
    year  = 0, # 年
    month = 1, # 月
    day   = 2, # 日
    st_ja = 3, # 都道府県
    st_en = 4, # state
    posis = 5, # 陽性者数
    tests = 6, # 検査数
    hospi = 7, # hospitalized
    serio = 8, # serious
    disch = 9, # discharged
    death =10, # deaths
    ern   =11, # Effective Reproduction Number
)

In [ ]:
dt_start = dt.strptime("2020/2/11", "%Y/%m/%d") # 統計開始日
dt_now   = dt.now() # 現在日時
dt_delta = (dt_now - dt_start).days # 差分
n_states = 47 # 都道府県数
poss = [] # 陽性者数
for i in np.arange(dt_delta):
    poss.append(np.zeros(n_states+1).tolist())
    poss[i][0] = dt_start + td(days=int(i))

In [ ]:
with codecs.open("prefectures.csv", encoding='utf-8') as f:
    l = f.readline()
    cnt = 0
    while l:
        l = f.readline().replace("\r\n", "").rstrip()
        elems = l.split(',')
        if len(elems) < 2:
            continue
        dt_ts = dt(int(elems[0]), int(elems[1]), int(elems[2]))
        if dt_ts < dt(2020, 2, 11, 0, 0, 0, 0): continue
        dt_idx  = (dt_ts - dt_start).days
        st_name = elems[offsets['st_en']]
        st_idx  = populations[st_name]['code']
        poss[dt_idx][st_idx] = blank2zero(elems[offsets['posis']])
        #if dt_idx > 0 and poss[dt_idx - 1][st_idx] > 0:
        #    poss[dt_idx][st_idx] = 
        poss[dt_idx][0] = dt_ts
        cnt += 1
print("Processed {} lines.".format(cnt))

In [ ]:
np_poss = np.array(poss)
# 累計値を新規数に変換する
data_os = 0
while data_os < 50:
    #print(np_poss[-(1 + data_os), :])
    if np_poss[-(1 + data_os), 1:].sum() > 0:
        break
    data_os += 1
#print("data offset is :{}".format(data_os))
if data_os != 0:
    np_diff = np_poss[1:-data_os] - np_poss[:-(1 + data_os)]
    np_diff[:, 0] = np_poss[1:-data_os, 0]
else:
    np_diff = np_poss[1:] - np_poss[:-1]
    np_diff[:, 0] = np_poss[1:, 0]
#print(np_diff[-1])

In [ ]:
#populations
tmpname = "prefectures.csv.tmp2"
with open(tmpname, "rt") as f:
    prev_line = int(f.read())
if prev_line == cnt:
    print("nothing to do.")
    if 'ipy' not in sys.argv[0]:
        sys.exit()
with open(tmpname, "wt") as f:
    f.write("{}".format(cnt))

In [ ]:
pref_info = {
    'Tokyo': { 'lid': "s47662", 'cname': "東京"    , 'pcode': populations['Tokyo']['code'] },
    'Kanagawa': { 'lid': "s47670", 'cname': "横浜" , 'pcode': populations['Kanagawa']['code']},
    'Aichi': { 'lid': "s47636", 'cname': "名古屋"  , 'pcode': populations['Aichi']['code'] },
    'Osaka': { 'lid': "s47772", 'cname': "大阪"    , 'pcode': populations['Osaka']['code'] },
    'Fukuoka': {'lid': "s47807", 'cname': "福岡"   , 'pcode': populations['Fukuoka']['code'] },
    'Okinawa': {'lid': "s47936", 'cname': "那覇"   , 'pcode': populations['Okinawa']['code'] }
}
#pref_info

In [ ]:
weather_info = {}
for k, v in pref_info.items():
    #print(k.lower(), v)
    csvname = "jma-{}.csv".format(k.lower())
    save_jma_data(csvname, city_code=v['lid'])
    weather_info[k] = parse_jma_csv(csvname)
    print("Loaded {} data.".format(len(weather_info[k])))
    time.sleep(2)
    if False:
        df_weather = pd.DataFrame(
            weather_info[k],
            columns=['Date', 'Temp', 'RH', 'VP', 'AP', 'AH', 'Fd'])
        fig = px.scatter(df_weather, x='Date', y=['AH'])
        fig.update_layout(template='plotly_dark')
        if "ipy" in sys.argv[0]:
            fig.show()

In [ ]:
def generate_rel_graph(
    x, y, txts, szs, clrs, pref,
    xtype, ytype, xlabel, ylabel, imgname, htmname):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        mode='markers', x=x, y=y, text=txts,
        marker=dict(
            opacity=0.85, size=szs, color=clrs,
            colorscale=[[0, 'rgb(59, 70, 222)'],
                        [.3, 'rgb(255, 255, 255)'],
                        [1, 'rgb(178, 10, 28)']],
            colorbar_title = '人数移動平均')))
    fig.update_layout(template='plotly_dark')
    tw_body = '{} {}/{}と感染日 ('.format(pref, xtype, ytype) + \
        today_str[:13] + '時)'
    fig.update_layout(
        height=500, width=500, margin=dict(l=12, r=5, b=12, t=42),
        xaxis =dict(showgrid=True, title=xlabel),
        yaxis =dict(title=ylabel),
        title=tw_body, showlegend=False)
    show_and_save_plotly(fig, imgname, js=False, show=False, image=True, html=True)
    tw_body += " (2020/5/25以降) "
    tw_body += " https://geneasyura.github.io/cov19-hm/{} ".format("{}.html".format(htmname))
    if False:#ytype == "絶対湿度":
        tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
def generate_rel_contour(
    x, y, txts, szs, clrs, pref,
    xtype, ytype, xlabel, ylabel, imgname, htmname):
    fig = go.Figure()
    fig.add_trace(go.Contour(
        x=x, y=y, z=clrs, text=txts,
        colorbar_title='感染人数',
        contours=dict(
            coloring='heatmap', showlabels=True,
            labelfont=dict(size=12, color='white')
        )))
    
    fig.add_trace(go.Scatter(
        mode='markers', x=x, y=y, text=txts,
        marker_line_width=1, marker_line_color="black",
        marker=dict(
            opacity=0.8, size=szs, color=clrs,
            colorscale=[[0, 'rgb(0, 255, 0)'],
                        [.4, 'rgb(128, 128, 0)'],
                        [1, 'rgb(255, 0, 0)']])))
    
    fig.update_layout(template='plotly_dark')
    tw_body = '{} 感染日の{}/{} ('.format(pref, xtype, ytype) + \
        today_str[:13] + '時)'
    fig.update_layout(
        height=500, width=500, margin=dict(l=12, r=5, b=12, t=42),
        xaxis =dict(domain=[0, 1], showgrid=False, title=xlabel,
                    range=[min(x), max(x)]),
        yaxis =dict(domain=[0, 1], showgrid=False, title=ylabel,
                    range=[min(y), max(y)]),
        title=tw_body, showlegend=False)
    show_and_save_plotly(fig, imgname, js=False, show=False, image=True, html=True)
    tw_body += " (2020/5/25以降) "
    tw_body += " https://geneasyura.github.io/cov19-hm/{} ".format("{}.html".format(htmname))
    if ytype == "絶対湿度":
        tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
dofs = 7 # = 報告日 - 感染日

In [ ]:
tw = get_twitter()

In [ ]:
for k, v in pref_info.items():
    pname = populations[k]['ja']
    cname = v['cname']
    pcode = v['pcode']
    print("Processing({}) {}:{} ...".format(pname, cname, pcode))
    np_winfo = np.array(weather_info[k])
    p_mv = moving_average(np_diff[:, pcode]) # 新規感染者数 移動平均
    x_mv = moving_average(np_winfo[:, 1]) # 平均気温 移動平均
    y_mv = moving_average(np_winfo[:, 5]) # 絶対湿度 移動平均
    z_mv = moving_average(np_winfo[:, 2]) # 相対湿度 移動平均
    pdays = np_diff[:, 0] # 新規感染者数 日付
    wdays = np_winfo[:, 0] # 気象データ 日付
    mv_max = np.max(p_mv)
    # 詰め替え
    sizes = []; xbins = []; ybins = [];
    zbins = []; colors = []; texts = []
    pi = 0
    for d in pdays: 
        pi += 1
        if d < dt(2021, 1, 1, 0, 0, 0, 0): continue
        # 1週間前の気象情報を参照する
        wi = int(np.where(wdays == (d - td(days=dofs)))[0])
        before1w = "{}".format(d - td(days=dofs))[:10]
        pv = p_mv[pi-1]
        msg = "%s =%d/%d p:%d t:%.2f ah:%.2f rh:%.2f" % \
        (before1w, wi, pi, pv, x_mv[wi], y_mv[wi], z_mv[wi])
        #print(msg)
        xbins.append(x_mv[wi]); ybins.append(y_mv[wi]); zbins.append(z_mv[wi]);
        colors.append(pv); sizes.append(max(5, (pv*10)/mv_max))
        texts.append("%s: %.1f人" % (before1w, pv))
    generate_rel_graph(
        xbins, ybins, texts, sizes, colors, "{}({})".format(pname, cname), "気温", "絶対湿度",
        '{} 平均気温 移動平均 [℃]'.format(cname),
        '{} 容積絶対湿度 移動平均 [g/㎥]'.format(cname),
        "{}-tvh.jpg".format(k.lower()), "{}-tvh".format(k.lower()))
    generate_rel_graph(
        xbins, zbins, texts, sizes, colors, "{}({})".format(pname, cname), "気温", "相対湿度",
        '{} 平均気温 移動平均 [℃]'.format(cname),
        '{} 相対湿度 移動平均 [%RH]'.format(cname),
        "{}-trh.jpg".format(k.lower()), "{}-tvh".format(k.lower()))
    generate_rel_contour(
        xbins, ybins, texts, sizes, colors, "{}({})".format(pname, cname), "気温", "絶対湿度",
        '{} 平均気温 移動平均 [℃]'.format(cname),
        '{} 容積絶対湿度 移動平均 [g/㎥]'.format(cname),
        "{}-tvh-cont.jpg".format(k.lower()), "{}-tvh".format(k.lower()))
    generate_rel_contour(
        xbins, zbins, texts, sizes, colors, "{}({})".format(pname, cname), "気温", "相対湿度",
        '{} 平均気温 移動平均 [℃]'.format(cname),
        '{} 相対湿度 移動平均 [%RH]'.format(cname),
        "{}-trh-cont.jpg".format(k.lower()), "{}-tvh".format(k.lower()))
    #break # TODO: 